In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import scikitplot as skplt

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "E:\DevPack\anaconda3\envs\data_science\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Player Ghost\AppData\Local\Temp\ipykernel_18600\4141141440.py", line 1, in <cell line: 1>
    import pandas as pd
  File "E:\DevPack\anaconda3\envs\data_science\lib\site-packages\pandas\__init__.py", line 11, in <module>
    __import__(dependency)
  File "E:\DevPack\anaconda3\envs\data_science\lib\site-packages\pytz\__init__.py", line 19, in <module>
    from pytz.lazy import LazyDict, LazyList, LazySet  # noqa
  File "E:\DevPack\anaconda3\envs\data_science\lib\site-packages\pytz\lazy.py", line 118, in <module>
    LazyList._props = [prop for prop in LazyList._props if hasattr(list, prop)]
MemoryError: Out of memory interning an attribute name

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\Dev

# Carregando base de dados  pré-processada

In [ ]:
database = pd.read_csv("../../Database/dataBaseWithNER.csv")

database = database.drop(columns=["Unnamed: 0"])
database = database.dropna()
target = database["target"].array
database

In [ ]:
emailsText = []
for email in database["email"]:
    emailsText.append(email)

In [ ]:
print(len(emailsText))

# Representação vetorial TFIDF

In [ ]:
tfidfVectorizer = TfidfVectorizer(analyzer="word",max_features=2100)

tfidfTransform = tfidfVectorizer.fit_transform(emailsText)

print(tfidfTransform)

In [ ]:
tfidfLabels = tfidfVectorizer.get_feature_names()
tfidfLabels

In [ ]:
dfTfidf = pd.DataFrame(data=tfidfTransform.toarray(), columns=tfidfLabels)

In [ ]:
dfTfidf

# Visualização de dados com TSNE

In [ ]:
model = TSNE(n_components=2, random_state=0)
array_red = model.fit_transform(dfTfidf)

df_tsne = pd.DataFrame(array_red)

df_tsne['Target'] = target
df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]

df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]

plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')

plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')

plt.title('Dados')
plt.xlabel('x')
plt.ylabel('y')

plt.show()

# Validação

In [ ]:
def getModel():
    return MultinomialNB()

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(dfTfidf.values,target,test_size=0.2)
modelo = getModel().fit(X_treino,y_treino)
score = modelo.score(X_teste,y_teste)
score

In [ ]:
scores = cross_val_score(getModel(),dfTfidf.values,target,cv=10)

scores.mean()

In [ ]:
predicoes = cross_val_predict(getModel(), dfTfidf.values, target, cv=10)

In [ ]:
print("\nClassification Report : ")
print(classification_report(target, predicoes, target_names=["Ham", "Spam"]))

In [ ]:
categories = ["Ham", "Spam"]

skplt.metrics.plot_confusion_matrix(
    [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
    title="Confusion Matrix",
    cmap="Purples",
    hide_zeros=True,
    figsize=(5,5)
)

plt.xticks()

In [ ]:
skplt.metrics.plot_confusion_matrix(
    [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
    normalize=True,
    title="Confusion Matrix",
    cmap="Purples",
    hide_zeros=True,
    figsize=(5,5)
)

plt.xticks()